In [42]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle


In [2]:
jan = pd.read_parquet('fhv_tripdata_2021-01.parquet')
feb = pd.read_parquet('fhv_tripdata_2021-02.parquet')

In [3]:
jan.shape

(1154112, 7)

Q1 -> 1154112

In [51]:
jan['duration_s']=(jan.dropOff_datetime-jan.pickup_datetime).dt.total_seconds()
feb['duration_s']=(feb.dropOff_datetime-feb.pickup_datetime).dt.total_seconds()

In [20]:
(jan.duration_s).mean()/60

19.167224093791013

Q2 -> 19.16

In [52]:
jan['duration_minutes'] = jan.duration_s/60 
feb['duration_minutes'] = feb.duration_s/60 

In [30]:
jan[(jan.duration_minutes>=1)&(jan.duration_minutes<=60)].shape

(1109826, 9)

In [31]:
jan.shape

(1154112, 9)

In [32]:
1154112 - 1109826

44286

Number of dropped records 44286

In [53]:
jan_final = jan[(jan.duration_minutes>=1)&(jan.duration_minutes<=60)]
feb_final = feb[(feb.duration_minutes>=1)&(feb.duration_minutes<=60)]

In [34]:
jan_final.describe()

,PUlocationID,DOlocationID,duration_s,duration_minutes
count,182818.000000,961919.000000,1.109826e+06,1.109826e+06
mean,139.861633,135.689147,9.748352e+02,1.624725e+01
std,74.763788,80.349678,6.930901e+02,1.155150e+01
min,1.000000,1.000000,6.000000e+01,1.000000e+00
25%,75.000000,67.000000,4.710000e+02,7.850000e+00
50%,146.000000,132.000000,7.940000e+02,1.323333e+01
75%,206.000000,213.000000,1.288000e+03,2.146667e+01
max,265.000000,265.000000,3.600000e+03,6.000000e+01


In [36]:
jan_final.isna().sum()

dispatching_base_num            0
pickup_datetime                 0
dropOff_datetime                0
PUlocationID               927008
DOlocationID               147907
SR_Flag                   1109826
Affiliated_base_number        773
duration_s                      0
duration_minutes                0
dtype: int64

In [38]:
jan_final.PUlocationID = jan_final.PUlocationID.fillna("-1")
jan_final.DOlocationID = jan_final.DOlocationID.fillna("-1")

C:\Users\Veera Vignesh\AppData\Local\Temp\ipykernel_1172\2809167046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_final.PUlocationID = jan_final.PUlocationID.fillna("-1")
C:\Users\Veera Vignesh\AppData\Local\Temp\ipykernel_1172\2809167046.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_final.DOlocationID = jan_final.DOlocationID.fillna("-1")


In [54]:
feb_final.PUlocationID = feb_final.PUlocationID.fillna("-1")
feb_final.DOlocationID = feb_final.DOlocationID.fillna("-1")

C:\Users\Veera Vignesh\AppData\Local\Temp\ipykernel_1172\150310745.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feb_final.PUlocationID = feb_final.PUlocationID.fillna("-1")
C:\Users\Veera Vignesh\AppData\Local\Temp\ipykernel_1172\150310745.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feb_final.DOlocationID = feb_final.DOlocationID.fillna("-1")


In [40]:
jan_final.PUlocationID.value_counts()

-1       927008
221.0      8330
206.0      6797
129.0      5379
115.0      4082
          ...  
111.0         5
27.0          4
34.0          3
2.0           2
110.0         1
Name: PUlocationID, Length: 262, dtype: int64

In [41]:
927008/jan_final.shape[0]

0.8352732770722617

Q3 -> 83%

In [44]:
cat = ['PUlocationID','DOlocationID']
jan_final[cat] = jan_final[cat].astype('str')

C:\Users\Veera Vignesh\AppData\Local\Temp\ipykernel_1172\662191961.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_final[cat] = jan_final[cat].astype('str')


In [55]:
feb_final[cat] = feb_final[cat].astype('str')


C:\Users\Veera Vignesh\AppData\Local\Temp\ipykernel_1172\4159566965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feb_final[cat] = feb_final[cat].astype('str')


In [45]:
train_dicts = jan_final[cat].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [56]:
val_dicts = feb_final[cat].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [46]:
X_train.shape

(1109826, 525)

Q4 -> 525

In [49]:
target = 'duration_minutes'
y_train = jan_final[target].values

In [57]:
y_val = feb_final[target].values

In [50]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519403716007

Q5 -> 10.52

In [58]:
y_val_pred = lr.predict(X_val)

In [59]:
mean_squared_error(y_val, y_val_pred, squared=False)

11.014287719752

Q6 -> 11.01